# Maîtriser l’utilisation des serveurs de calcul
## Les espaces de stockage
Il y a plusieurs types de stockage à NOIRN :
* Personnel (`/home`)
* Temporaire local (`$SLURM_TMPDIR`)
* Temporaire réseau (`/scratch`)
* Projet partagé (`/project`)
* *Nearline* de longue durée (`/nearline`)
* [Dépôt fédéré de données de recherche](https://www.frdr-dfdr.ca/repo/) (DFDR)

Or, vos données ont plusieurs aspects :
* **Taille** : petits, gros, très gros fichiers
* **Nombre** : peu nombreux ou en très grand nombre
* **Transférabilité** : données regroupées et/ou compressées
* **Vie** : pendant une tâche, entre des tâches, d'un projet à l'autre, à archiver
* **Niveau d'accès** : données confidentielles, partagées, publiées

Le but de ce chapitre est d'analyser différents scénarios concernant le stockage.

## Description de vos données
Au chapitre précédent, nous avons vu comment obtenir le nombre de fichiers de nos données locales, de même que la taille totale.

Sur une grappe de calcul, le système de fichiers parallel [*Lustre*](https://docs.computecanada.ca/wiki/Tuning_Lustre/fr) est surtout optimisé pour des fichiers de plus de 10 Mo, c'est-à-dire pour des accès à large bande-passante. Ainsi, à l'échelle des grappes de calcul :

* 1 Ko - 100 Ko : très petit fichier
* 1 Mo - 10 Mo : petit fichier
* 100 Mo - 1 Go : taille raisonnable
* 10 Go - 100 Go : gros fichier
* 1 To - 10 To : très gros fichier

À quantité totale égale d'octets, il est plus difficile pour *Lustre* de gérer un grand nombre de petits fichiers qu'un plus petit nombre de grands fichiers. Ainsi, du point de vue du système de stockage :

* Par dossier :
  * 1 à 100 fichiers par dossier : c'est raisonnable
  * 1000 à 5k fichiers par dossier : des lenteurs se font sentir
  * 10k fichiers et plus : il faut classer les fichiers dans des sous-dossiers
* Par espace limité par un quota :
  * 1 à 500k fichiers : sauvegarde facilement à jour
  * 1M à 5M fichiers : sauvegarde potentiellement plus longue que 24h
  * 10M fichiers et plus : sauvegarde plus longue que 24h

### Transférabilité
#### Regroupement dans un fichier archive
Étant donné que le transfert ou la synchronisation de [plusieurs petits fichiers](https://docs.computecanada.ca/wiki/Handling_large_collections_of_files/fr) implique un accès fréquent aux métadonnées du système de stockage Lustre, il est donc préférable de [regrouper les fichiers](https://docs.computecanada.ca/wiki/Archiving_and_compressing_files/fr) d'un jeu de données avant tout transfert.
Pour ce faire, il existe plusieurs formats décodables sur les différentes grappes :

* WinZip ou Zip (`.zip`) - compressé
* 7-Zip (`.7z`) - compressé
* Tape ARchive ou [TAR](https://docs.computecanada.ca/wiki/A_tutorial_on_%27tar%27/fr) (`.tar`) - non compressé, sauf si utilisé avec Gzip ou autre
* Disk ARchive ou [DAR](https://docs.computecanada.ca/wiki/Dar/fr) (`.dar`) - compression configurable

#### Compression
La [compression des fichiers](https://docs.computecanada.ca/wiki/Archiving_and_compressing_files/fr) permet :
1. d'économiser de l'espace-disque et
1. de réduire la quantité d'octets à transférer.

Or, la compression a aussi un **coût en temps CPU** à chaque fois que nous voulons lire les données ou les écrire.
* Ainsi, il vaut mieux utiliser la compression lorsque c'est *avantageux* en espace et/ou en temps.
  * Par exemple : décompression des fichiers sur le noeud de calcul plutôt que de lire le fichier de *Lustre*.

Concernant les fichiers binaires tels que FLAC, JPEG, PNG, etc., leur encodage inclut déjà une compression des données.
* Selon le type de données que vous utilisez dans votre domaine de recherche, il se pourrait qu'elles soient déjà compressées. À vous de vérifier.
* En pratique, on gagne peu d'espace à compresser davantage ces fichiers.

Sur les grappes de calcul, il y a deux types de compression :
1. Compression explicite :
  * Lorsque vous utilisez un outil de compression (`7z`, `bzip2`, `gzip`, `zip`, etc.)
  * Lorsque l'encodage permet de compresser les données (`dar`, bibliothèque `zlib`, etc.)
  * Lors d'un transfert avec `scp` ou `rsync`, il existe des options permettant de compresser automatiquement les données lors du transfert. Cela étant dit, le transfert risque d'être limité par la vitesse du processeur...
1. Compression implicite :
  * Lorsque le système de fichiers compresse automatiquement vos données avant de les écrire sur le système de stockage
  * La commande `du` montre l'espace réellement utilisée sur le disque
  * La commande `du --apparent-size` montre la taille normale du fichier

#### Fichiers épars (sparse files)
Restez à l'affût des fichiers épars dont la faible taille sur le disque n'est pas représentative de la taille normale du fichier :
* Une certaine taille `T` est allouée pour le fichier. Par exemple : 1 To
* Seulement `t` octets sont réellement écrits dans le fichier (où `t << T`). Par exemple : 1 Go
* Ce type de fichiers est typiquement utilisé pour des volumes de stockage de machines virtuelles, ce qui économise de l'espace sur le serveur hôte
* Problème : lors d'un transfert ou d'une sauvegarde (backup), ce sont les `T` octets qui sont lus, ce qui peut être catastrophique si `T` est vraiment très grand
* Solution : les fichiers `.dar` permettent d'optimiser l'encapsulation de fichiers épars avant un transfert ou une copie des données

#### Exercice
Dans un terminal Jupyter, essayez les commandes :
* `du -s ~`
* `du -s --apparent-size ~`

* Notez les différences, s'il y en a
* Notez à quel point cette commande peut surcharger le système de stockage; le survol récursif des fichiers constitue un grand nombre d'accès aux métadonnées de *Lustre*

En pratique, les grappes de calcul offrent des outils efficaces pour mesurer l'espace utilisé en fonction de l'espace alloué.

### Plan de gestion des données
* Vie des données
* Niveaux d'accès

## Les différents types de stockage
* Stockage accessibles sur les noeuds de calcul
* Stockage non-accessible sur les noeuds de calcul

### Votre dossier personnel (`/home`)

### Dossier temporaire local (`$SLURM_TMPDIR`)

### Espace réseau pour données temporaires (`/scratch`)

### Espace projet partagé (`/project`)
* Configuration des ACLs

### Stockage très longue durée (`/nearline`)

### Dépot de données de recherche (DFDR)

## Gestion du stockage
* `diskusage_report` / `quota`
* Consommation de l'espace `/project` par utilisateur
* `diskusage_explorer`
* Accès aux données d'une personne ayant quitté le groupe